


# GravixLayer AgentBox - Connecting to LLM

This notebook:
1. Asks you for a prompt
2. Sends it to an AI model using GravixLayer
3. Saves the AI response into a text file **inside a sandbox**
4. Downloads that file back to the local environment so you can download it from the notebook UI

## Prerequisites

1. Install the GravixLayer SDK: `pip install gravixlayer`
2. Set your API key as an environment variable: `GRAVIXLAYER_API_KEY`

In [1]:
# Install GravixLayer SDK if not already installed
!pip install gravixlayer


In [2]:
import os
from gravixlayer import GravixLayer

# Make sure GRAVIXLAYER_API_KEY is set in your environment before running this cell
client = GravixLayer()

# Choose the model you want to demo
MODEL_NAME = "meta-llama/llama-3.1-8b-instruct"


In [3]:
# 1) Get user prompt
user_prompt = input("Enter your prompt for the AI: ")

# 2) Call the AI model via GravixLayer
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_prompt},
    ],
)

ai_response = completion.choices[0].message.content
print("\nAI response:\n")
print(ai_response)



AI response:

Here's a poem on love:

Love's gentle touch ignites the flame,
A warmth that spreads, a heart that claims,
A feeling deep, a soul that's true,
A connection strong, a love that shines through.

In eyes that meet, a spark takes flight,
A whispered promise, a love so bright,
A tender caress, a gentle kiss,
A love that grows, a heart that misses.

With every breath, a love is born,
A flame that flickers, a heart that yearns,
For more of you, for more of me,
A love that's pure, a love that's free.

Through laughter and tears, through joy and pain,
Love stands strong, a love that remains,
A bond that's unbreakable, a tie that's true,
A love that's pure, a love that shines through.

In your arms, I find my home,
A place where love resides, a place to roam,
With you by my side, I feel complete,
A love that's real, a love that's sweet.

So let us cherish, let us adore,
This love we share, this love we explore,
For it's a gift, a treasure so rare,
A love that's ours, a love that's

In [ ]:
from datetime import datetime, timezone

# 3) Create a sandbox and write the AI response to a file inside it
sandbox = client.sandbox.sandboxes.create(
    provider="gravix",
    region="eu-west-1",
    template="python-base-v1",
)

try:
    sandbox_path = "/home/user/ai_response.txt"

    # Prepare content with some metadata (timezone-aware UTC)
    content = (
        "Prompt: " + user_prompt + "\n"
        "Generated at: " + datetime.now(timezone.utc).isoformat() + "\n\n"
        "AI Response:\n" + ai_response + "\n"
    )

    # Write text file into the sandbox
    client.sandbox.sandboxes.write_file(
        sandbox.sandbox_id,
        path=sandbox_path,
        content=content,
    )
    print(f"Wrote AI response to sandbox file: {sandbox_path}")

    # 4) Download the file from sandbox to local environment
    file_response = client.sandbox.sandboxes.read_file(
        sandbox.sandbox_id,   
        path=sandbox_path,
    )

    # Unwrap FileReadResponse -> get the actual content
    if hasattr(file_response, "content"):
        downloaded_content = file_response.content
    else:
        downloaded_content = file_response

    # If it’s bytes, decode to str
    if isinstance(downloaded_content, bytes):
        downloaded_content = downloaded_content.decode("utf-8")

    local_filename = "ai_response.txt"
    with open(local_filename, "w", encoding="utf-8") as f:
        f.write(downloaded_content)

    print(f"Downloaded sandbox file to local file: {local_filename}")
    print("You can now download 'ai_response.txt' from the notebook file browser.")

finally:
    # Always clean up the sandbox
    client.sandbox.sandboxes.kill(sandbox.sandbox_id)
    print("Sandbox killed.")


Wrote AI response to sandbox file: /home/user/ai_response.txt
Downloaded sandbox file to local file: ai_response.txt
You can now download 'ai_response.txt' from the notebook file browser.
Sandbox killed.
